In [1]:
import pandas as pd
import torch
from tqdm import tqdm 

In [2]:
tdf = pd.read_json("../dataset/traindf.json")
vdf = pd.read_json("../dataset/valdf.json")

In [3]:
def returnSynthSentences(df):
    csnliHinglish = list(df["Hinglish_csnliop"])
    sentences = []
    for line in csnliHinglish:
        sent = []
        for token in line:
            sent.append(token[1])
        sent = " ".join(sent)
        sentences.append(sent)
    
    return sentences

In [4]:
def returnHumSentences(df):
    csnliHinglish = list(df["hum_gen_csnliop"])
    sentences = []
    for sents in csnliHinglish:
        ref = []
        for line in sents:
            sent = []
            for token in line:
                sent.append(token[1])
            sent = " ".join(sent)
            ref.append(sent)
            
        sentences.append(ref)
    
    return sentences

In [5]:
trainSynth = returnSynthSentences(tdf)
validSynth = returnSynthSentences(vdf)

trainHuman = returnHumSentences(tdf)
validHuman = returnHumSentences(vdf)

print(len(trainSynth), len(trainHuman), len(validSynth), len(validHuman))

2766 2766 395 395


In [6]:
xlmr = torch.hub.load('pytorch/fairseq', 'xlmr.large').cuda()
xlmr.eval()

Downloading: "https://github.com/pytorch/fairseq/archive/main.zip" to /home/prashantk/.cache/torch/hub/main.zip
fatal: Not a git repository (or any parent up to mount point /home)
Stopping at filesystem boundary (GIT_DISCOVERY_ACROSS_FILESYSTEM not set).


running build_ext
cythoning fairseq/data/data_utils_fast.pyx to fairseq/data/data_utils_fast.cpp
cythoning fairseq/data/token_block_utils_fast.pyx to fairseq/data/token_block_utils_fast.cpp
building 'fairseq.libbleu' extension
creating /home/prashantk/.cache/torch/hub/pytorch_fairseq_main/build
creating /home/prashantk/.cache/torch/hub/pytorch_fairseq_main/build/temp.linux-x86_64-3.7
creating /home/prashantk/.cache/torch/hub/pytorch_fairseq_main/build/temp.linux-x86_64-3.7/fairseq
creating /home/prashantk/.cache/torch/hub/pytorch_fairseq_main/build/temp.linux-x86_64-3.7/fairseq/clib
creating /home/prashantk/.cache/torch/hub/pytorch_fairseq_main/build/temp.linux-x86_64-3.7/fairseq/clib/libbleu
Emitting ninja build file /home/prashantk/.cache/torch/hub/pytorch_fairseq_main/build/temp.linux-x86_64-3.7/build.ninja...
Compiling objects...
Allowing ninja to set a default number of workers... (overridable by setting the environment variable MAX_JOBS=N)
[1/2] c++ -MMD -MF /home/prashantk/.cach

2022-05-08 20:34:47 | INFO | fairseq.file_utils | http://dl.fbaipublicfiles.com/fairseq/models/xlmr.large.tar.gz not found in cache, downloading to /tmp/tmpqd25_qcm
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1028340964/1028340964 [01:08<00:00, 14918542.38B/s]
2022-05-08 20:35:56 | INFO | fairseq.file_utils | copying /tmp/tmpqd25_qcm to cache at /home/prashantk/.cache/torch/pytorch_fairseq/3f864e15bb396f062dd37494309dbc4238416edd1f8ef293df18b1424813f2fe.cf46c7deb6b9eaa3e47c17b9fc181669c52bc639c165fbc69166a61487662ac9
2022-05-08 20:36:06 | INFO | fairseq.file_utils | creating metadata file for /home/prashantk/.cache/torch/pytorch_fairseq/3f864e15bb396f062dd37494309dbc4238416edd1f8ef293df18b1424813f2fe.cf46c7deb6b9eaa3e47c17b9fc181669c52bc639c165fbc69166a61487662ac9
2022-05-08 20:36:06 | INFO | fairseq.file_utils | removing temp file /tmp/tmpqd25_qcm
2022-05-08 20:36:06 | I

RobertaHubInterface(
  (model): RobertaModel(
    (encoder): RobertaEncoder(
      (sentence_encoder): TransformerEncoder(
        (dropout_module): FairseqDropout()
        (embed_tokens): Embedding(250002, 1024, padding_idx=1)
        (embed_positions): LearnedPositionalEmbedding(514, 1024, padding_idx=1)
        (layernorm_embedding): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (layers): ModuleList(
          (0): TransformerEncoderLayerBase(
            (self_attn): MultiheadAttention(
              (dropout_module): FairseqDropout()
              (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
            )
            (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
            (d

In [7]:
def returnSentenceVectors(sents):
    with torch.no_grad():
        vectors = []
        for line in tqdm(sents):
            if isinstance(line, list):
                refs = []
                for st in line:
                    tokens = xlmr.encode(st.strip())
                    llf = xlmr.extract_features(tokens)
                    llf = llf.mean(dim=1)
                    refs.append(llf)
                refs = torch.cat(refs, dim=0)
                refs = refs.mean(dim=0).unsqueeze(0)
                vectors.append(refs)

            else:
                tokens = xlmr.encode(line)
                llf = xlmr.extract_features(tokens)
                llf = llf.mean(dim=1)
                vectors.append(llf)

        vectors = torch.cat(vectors, dim=0)
        return vectorsL

In [8]:
trainSynthVecs = returnSentenceVectors(trainSynth)
validSynthVecs = returnSentenceVectors(validSynth)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 395/395 [00:06<00:00, 58.99it/s]


In [9]:
torch.save(trainSynthVecs, "train.synth.pt")
torch.save(validSynthVecs, "valid.synth.pt")

In [10]:
trainHumanVecs = returnSentenceVectors(trainHuman)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2766/2766 [01:53<00:00, 24.34it/s]


In [11]:
validHumanVecs = returnSentenceVectors(validHuman)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 395/395 [00:16<00:00, 24.04it/s]


In [12]:
torch.save(trainHumanVecs, "train.human.pt")
torch.save(validHumanVecs, "valid.human.pt")